In [1]:
datasets_path = 'eepy/datasets'
hf_cache_path = 'eepy/hf-cache'

# Llava
https://huggingface.co/docs/transformers/model_doc/llava

In [2]:
from PIL import Image
import torch
import requests
from transformers import AutoProcessor, LlavaForConditionalGeneration

In [ ]:
url_llava = "llava-hf/llava-1.5-7b-hf"
# processes img
processor = AutoProcessor.from_pretrained(url_llava, cache_dir=hf_cache_path)
# actual model 
model = LlavaForConditionalGeneration.from_pretrained(url_llava, cache_dir=hf_cache_path)

model.safetensors.index.json:   0%|          | 0.00/70.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

In [19]:
print('Enter your question.')
user_question = input()
print(f"Your question: {user_question}")

Enter your question.
Your question: What can you see on the images?


In [20]:
# sample img from internet
#url_img = "https://www.ilankelman.org/stopsigns/australia.jpg"
#image = Image.open(requests.get(url_img, stream=True).raw)

prompt = f"\tUSER: <image> {user_question} \n\tASSISTANT:"

path_img = "/mount/studenten/team-lab-cl/data2024/fm_med_vqa/momo_fm/FM_MedVQA/sample_datasets/ROCOv2/train/"
imgs = ["ROCOv2_2023_train_000307.jpg", "ROCOv2_2023_train_001490.jpg", "ROCOv2_2023_train_060077.jpg"]

for img in imgs:
    image = Image.open(path_img+img) 
    caption = "Describe the scene in this image."

    # Prepare the inputs
    inputs = processor(images=image, text=prompt, return_tensors="pt")

    # Generate predictions
    generate_ids = model.generate(**inputs, max_new_tokens=80)
    caption = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

    # Display the generated caption
    print("Generated Caption:\n", caption, "\n")

Generated Caption:
 	USER:   What can you see on the images? 
	ASSISTANT: The images show a close-up view of a person's abdomen, focusing on the stomach area. The stomach is visible, along with the spine and the ribcage. The images provide a detailed perspective of the internal organs and their surrounding structures. 

Generated Caption:
 	USER:   What can you see on the images? 
	ASSISTANT: The image shows a close-up view of a person's back, focusing on the spine and the area around the spine. There are two arrows pointing towards the spine, indicating the location of the image. The image is in black and white, giving it a classic or vintage appearance. 

Generated Caption:
 	USER:   What can you see on the images? 
	ASSISTANT: The images show a close-up view of a person's lungs, specifically focusing on the right lung. The lungs are visible in the chest area, and the right lung is shown in detail, highlighting its structure and function. The images provide a clear and detailed persp

## Results of trial runs:
#### Original captions: 
Img1 ROCOv2_2023_train_000307.jpg: *CT scan with sagittal view of inferior vena cava thrombus*   
Img2 ROCOv2_2023_train_001490.jpg: *Barium swallow examination showing evidence of multiple webs in the upper esophagus (arrows).*   
Img3 ROCOv2_2023_train_060077.jpg: *Chest computed tomography showing a right lung nodule (arrow).*  


#### Question 1: Is the body part healthy?  
Notes: max_new_tokens=30, to second sentence is cut for 2nd sample.  
Model Output:  

    Generated Caption:
        USER:   Is the body part healthy? 
        ASSISTANT: No 

    Generated Caption:
        USER:   Is the body part healthy? 
        ASSISTANT: No, the body part is not healthy. The image shows a close-up of a person's back, with a large, curved 

    Generated Caption:
        USER:   Is the body part healthy? 
        ASSISTANT: No


#### Question 2: Which body part is seen?
Notes: same question, different run gave 'abdomen' instead of 'stomach'.    
Model output:  

    Generated Caption:  
        USER:   Which body part is seen?   
        ASSISTANT: Stomach   

    Generated Caption:  
        USER:   Which body part is seen?   
        ASSISTANT: Spine   

    Generated Caption:  
        USER:   Which body part is seen?   
        ASSISTANT: Lung   


#### Question 3: What can you see on the images? 
Notes: max_new_tokens=80  
Model output:  

    Generated Caption:  
        USER:   What can you see on the images?   
        ASSISTANT: The images show a close-up view of a person's abdomen, focusing on the stomach area. The stomach is visible, along with the spine and the ribcage. The images provide a detailed perspective of the internal organs and their surrounding structures. 

    Generated Caption:  
        USER:   What can you see on the images?   
        ASSISTANT: The image shows a close-up view of a person's back, focusing on the spine and the area around the spine. There are two arrows pointing towards the spine, indicating the location of the image. The image is in black and white, giving it a classic or vintage appearance. 

    Generated Caption:  
        USER:   What can you see on the images?   
        ASSISTANT: The images show a close-up view of a person's lungs, specifically focusing on the right lung. The lungs are visible in the chest area, and the right lung is shown in detail, highlighting its structure and function. The images provide a clear and detailed perspective of the human respiratory system. 